# Quantum Sampling Regression (QSR)

QSR trades extra classical post-processing (Fourier-regression) for a **single** batch of quantum evaluations.  
Instead of iteratively calling VQE, it samples the cost
\begin{equation*}
  C(\boldsymbol\theta)=\langle\psi(\boldsymbol\theta)|\hat H|\psi(\boldsymbol\theta)\rangle
\end{equation*}
at a carefully chosen set of parameter points, fit its unknown Fourier coefficients
$\{a_0,a_k,b_k\}$ by least-squares, then find the global minimum.  
This approach requires only a single batch of quantum executions, after which all further processing is done classically—eliminating the need for repeated quantum–classical loops.

More informations can be found in the Qiskit course [Qiskit QSR](https://learning.quantum.ibm.com/course/variational-algorithm-design/instances-and-extensions#quantum-sampling-regression-qsr) and in the original paper [An optimal quantum sampling regression algorithm for variational eigensolving in the low qubit number regime](https://arxiv.org/abs/2012.02338).

In [1]:
from qiskit import QuantumCircuit
import numpy as np
from scipy.optimize import minimize
from qiskit.primitives import StatevectorEstimator
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp, Statevector, Operator
from qiskit.circuit import Parameter
import itertools

estimator = StatevectorEstimator()

In [2]:
def cost_func_qsr(parameters, ansatz, observable, estimator):
    estimator_job = estimator.run([(ansatz, observable, [parameters])])
    estimator_result = estimator_job.result()[0]

    cost = estimator_result.data.evs[0]
    return cost

In [3]:
def estimate_bandwidth_from_circuit(circuit):
    """
    Estimate the Fourier bandwidth S from a manually defined circuit.
    Works directly without decomposition.
    """
    S = 0
    entanglement_present = False

    for instr in circuit.data:
        name = instr.operation.name.lower()

        if name in ['rx', 'ry', 'rz']:
            S += 1
        elif name in ['cx', 'cz', 'iswap', 'cy']:
            entanglement_present = True

    if entanglement_present:
        S = max(2*S, S+1)

    return max(S, 1)


In [4]:
def sampler_qsr(parameters_list, ansatz, observable, estimator):
    """
    Sample the cost function over a list of parameter vectors.
    
    Args:
        parameters_list: list of parameter arrays (each array shape = (N_parameters,))
        ansatz: QuantumCircuit, parameterized ansatz
        observable: Hamiltonian (SparsePauliOp)
        estimator: Estimator primitive (e.g., StatevectorEstimator)
        
    Returns:
        cost_sampled: list of cost function values corresponding to the parameter list
    """
    
    cost_sampled = []
    
    for params in parameters_list:
        # params must be an array of all θ values [θ₁, θ₂, ..., θₙ]
        cost_value = cost_func_qsr(params, ansatz, observable, estimator)
        cost_sampled.append(cost_value)

    return cost_sampled


### Classical function to post-process the samples

In [5]:
def build_F_matrix(theta_samples, max_harmonic):
    theta_samples = np.array(theta_samples)
    N_samples, N_parameters = theta_samples.shape
    
    # Generate all combinations of frequencies for each parameter
    freq_range = list(range(0, max_harmonic + 1))  # 0,1,...,S
    all_freqs = list(itertools.product(freq_range, repeat=N_parameters))
    
    # Remove the trivial case where all frequencies are 0
    all_freqs.remove((0,) * N_parameters)
    
    # Initialize the F matrix
    N_basis = 1 + 2 * len(all_freqs)  # 1 constant + 2 (cos and sin) for each frequency combo
    F = np.zeros((N_samples, N_basis))
    
    # First column = constant term
    F[:, 0] = 1.0

    # Fill the F matrix
    for i, freqs in enumerate(all_freqs):
        # Compute the dot product k · theta for each sample
        argument = np.sum(freqs * theta_samples, axis=1)  # shape (N_samples,)
        
        F[:, 2*i + 1] = np.cos(argument)  # cosine term
        F[:, 2*i + 2] = np.sin(argument)  # sine term
        
    return F, all_freqs


In [6]:
def solve_fourier_coefficients(theta_samples, h_samples, max_harmonic):
    F, all_freqs = build_F_matrix(theta_samples, max_harmonic)
    h_samples = np.array(h_samples)
    
    # Solve least squares: minimize || F c - h ||
    c, residuals, rank, s = np.linalg.lstsq(F, h_samples, rcond=None)
    
    return c, all_freqs

In [7]:
def reconstructed_h(theta, coefficients, all_freqs):
    theta = np.array(theta)
    h = coefficients[0]  # constant term

    for i, freqs in enumerate(all_freqs):
        argument = np.dot(freqs, theta)  # dot product
        h += coefficients[2*i + 1] * np.cos(argument)
        h += coefficients[2*i + 2] * np.sin(argument)
    
    return h



In [8]:
def find_global_minimum(coefficients, all_freqs):
    N_parameters = len(all_freqs[0])  # inferred from freq tuples
    
    # Initial guess: center of the domain
    x0 = np.full(N_parameters, np.pi)

    # Bounds: θ ∈ [0, 2π] for each parameter
    bounds = [(0, 2*np.pi) for _ in range(N_parameters)]

    # Minimize
    result = minimize(
        lambda theta: reconstructed_h(theta, coefficients, all_freqs),
        x0=x0,
        bounds=bounds,
        method='L-BFGS-B'  # Good for bounded smooth problems
    )
    
    theta_min = result.x
    h_min = result.fun
    return theta_min, h_min

### Global function to run he QSR algorithm

In [9]:
def run_qsr(ansatz, hamiltonian, S, estimator):
    # 1) estimate max harmonic
    # S = estimate_bandwidth_from_circuit(ansatz)

    # 2) build the multi-D grid of theta-points
    num_params = len(ansatz.parameters)
    M = 2*S + 1
    ranges = [np.linspace(0, 2*np.pi, M, endpoint=False) for _ in range(num_params)]
    mesh   = np.meshgrid(*ranges, indexing='ij')
    thetas = np.stack([m.flatten() for m in mesh], axis=-1)  # shape (M^n, n)

    # 3) quantum sampling
    h_samples = sampler_qsr(thetas, ansatz, hamiltonian, estimator)

    # 4) regression to get Fourier coefficients
    coeffs, all_freqs = solve_fourier_coefficients(thetas, h_samples, S)

    # 5) classical minimization of the surrogate
    theta_min, h_min = find_global_minimum(coeffs, all_freqs)

    # 6) build the final Statevector U(θ*)|0⟩
    param_dict = {p: theta_min[i] for i, p in enumerate(ansatz.parameters)}
    final_circ = ansatz.assign_parameters(param_dict)
    sv_opt = Statevector(final_circ)

    return h_min, sv_opt


### Test Case 1 — 2-Qubit Non-degenerate Hamiltonian

**Hamiltonian**  
\begin{equation*}
H = Z\otimes Z \;+\; 2\,X\otimes X
\end{equation*}

**Spectrum**  
-3, -1, +1, +3

**Eigenstates**  
- $\displaystyle \ket{\psi_0} = \frac{1}{\sqrt2}\bigl(\ket{01} - \ket{10}\bigr)$  
- $\displaystyle \ket{\psi_1} = \frac{1}{\sqrt2}\bigl(\ket{00} - \ket{11}\bigr)$  
- $\displaystyle \ket{\psi_2} = \frac{1}{\sqrt2}\bigl(\ket{01} + \ket{10}\bigr)$  
- $\displaystyle \ket{\psi_3} = \frac{1}{\sqrt2}\bigl(\ket{00} + \ket{11}\bigr)$

We expect to find only the ground state

In [ ]:
# 1) Build the variational ansatz (Manual construction with 2 parameters)
theta0 = Parameter("θ0")
theta1 = Parameter("θ1")

ansatz1 = QuantumCircuit(2)
ansatz1.ry(theta0, 0)
ansatz1.ry(theta1, 1)
ansatz1.cx(0, 1)

# 2) Define the observable
obs1 = SparsePauliOp.from_list([
    ("ZZ", 1.0),
    ("XX", 2.0)
])

# 3) Estimate S
S_estimated = estimate_bandwidth_from_circuit(ansatz1)
# print(S_estimated)

# 4) Run QSR
eigenval_qsr, sv_qsr = run_qsr(ansatz1, obs1, S_estimated, estimator)

# 5) Fix global phase & format the statevector

def fix_global_phase(vec, threshold = 1e-3):
    v = vec.copy()
    # find first significant index
    for amp in v:
        if abs(amp) > threshold:
            phi = np.angle(amp)
            v *= np.exp(-1j * phi)
            if v[np.argmax(np.abs(v))].real < 0:
                v *= -1
            break
    return v

vec = fix_global_phase(sv_qsr.data)
threshold = 1e-3
terms = [
    f"({amp:.5f})|{format(idx, '02b')}⟩"
    for idx, amp in enumerate(vec)
    if abs(amp) > threshold
]

# 6) Print results
print(f"Test 1 QSR → eigenvalue = {eigenval_qsr:.6f}")
print("Statevector:", " + ".join(terms))


4


Test 1 QSR → eigenvalue = -3.000000
Statevector: (0.70711+0.00000j)|01⟩ + (-0.70711+0.00000j)|10⟩


QSR struggled on larger-qubit problems because our ansatz couldn’t span correctly the space. A simpler, more targeted ansatz that still covers the right eigenstates may make one-shot QSR practical for bigger systems.